In [1]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#clasificadores
from utils.clasificacion import *

#adaptacion
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

#otros
import os
import numpy as np
import pandas as pd
from sklearn.externals import joblib
import itertools

#variables para guardar los resultados
tipo = pruebas[3]

# Pruebas con el dataset Amazon (3000 Dimensiones)

In [2]:
dims = 3000
dataset_name = datasets[0]

In [3]:
print tipo
print dataset_name
print dims
print data_path

pca
amazon
3000
data


In [4]:
# cargando dataset
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [5]:
parametros = {
    'n_components': [int(dims/2), int(dims/3), int(dims/4)],
}

In [12]:
porcentajes = [25, 50, 100]
modelos = {}

pairs = list(itertools.permutations(domains, 2))

for src, tgt in pairs:
    modelos[(src, tgt)] = {}
    for ptje in porcentajes:
        modelos[(src, tgt)][ptje] = [100, None]

In [13]:
df = pd.DataFrame(columns=dataframe_columns+['Porcentaje'])

i=0
j=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

porcentajes = [25, 50, 100]


for src, tgt in pairs:
    print "Tarea %d de %d" % (i+1, tareas)
    tarea = src[0]+'->'+tgt[0]
    
    #datos de entrenamiento
    X_src = labeled[src]['X_tr'][:, :dims].todense()
    y_src = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
    
    X_tgt = labeled[tgt]['X_tr'][:, :dims].todense()
    y_tgt = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
    
    X_train = np.concatenate((X_src, X_tgt))
    
    #datos de prueba
    X_tgt_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    y_tgt_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
    
    
    #baseline in-domain error
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)
    
    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tgt, y_tgt)
    b_error = 1-svc.score(X_tgt_ts, y_tgt_ts)

    k = 0    
    for n_components in parametros['n_components']:
        print "\tEntrenando modelo de adaptacion %d" % k
        # entrenar un modelo de adaptacion
        modelo_temp = PCA(n_components=n_components)
        t_adaptar = modelo_temp.fit(X_train)

        # entrenar un clasificador con los datos adaptados
        X_src_a = modelo_temp.transform(X_src)
        clf = get_best_score(X_src_a, y_src, classifier='SVC', n_jobs=4)

        # se prueba con distintos porcentajes
        j = i*3
        for porcentaje in porcentajes:
            if porcentaje == 100:
                X_test_pc, y_test_pc = X_tgt_ts, y_tgt_ts
            else:
                X_test_pc, _, y_test_pc, _ = train_test_split(X_tgt_ts, y_tgt_ts, train_size=porcentaje/float(100))
            
            X_test_pc_a = modelo_temp.transform(X_test_pc)
            t_error = 1-clf.score(X_test_pc_a, y_test_pc)
            
            if t_error <= modelos[(src, tgt)][porcentaje][0]:
                modelos[(src, tgt)][porcentaje] = [t_error, modelo_temp]
                t_loss = t_error - b_error
                df.loc[j] = ['PCA', tarea, src, tgt, b_error*100, t_error*100, t_loss*100, porcentaje]
    
            j += 1
        k += 1
    i+=1

Tarea 1 de 12
Cargando modelo existente.
tEntrenando modelo de adaptacion 0
tEntrenando modelo de adaptacion 1
tEntrenando modelo de adaptacion 2
Tarea 2 de 12
Cargando modelo existente.
tEntrenando modelo de adaptacion 0
tEntrenando modelo de adaptacion 1
tEntrenando modelo de adaptacion 2
Tarea 3 de 12
Cargando modelo existente.
tEntrenando modelo de adaptacion 0
tEntrenando modelo de adaptacion 1
tEntrenando modelo de adaptacion 2
Tarea 4 de 12
Cargando modelo existente.
tEntrenando modelo de adaptacion 0
tEntrenando modelo de adaptacion 1
tEntrenando modelo de adaptacion 2
Tarea 5 de 12
Cargando modelo existente.
tEntrenando modelo de adaptacion 0
tEntrenando modelo de adaptacion 1
tEntrenando modelo de adaptacion 2
Tarea 6 de 12
Cargando modelo existente.
tEntrenando modelo de adaptacion 0
tEntrenando modelo de adaptacion 1
tEntrenando modelo de adaptacion 2
Tarea 7 de 12
Cargando modelo existente.
tEntrenando modelo de adaptacion 0
tEntrenando modelo de adaptacion 1
tEntrenando m

In [14]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss,Porcentaje
0,mSDA,e->d,electronics,dvd,15.762894,25.320513,9.557619,25
1,mSDA,e->d,electronics,dvd,15.762894,24.320802,8.557908,50
2,mSDA,e->d,electronics,dvd,15.762894,26.553164,10.790270,100
3,mSDA,e->k,electronics,kitchen,8.587715,8.480000,-0.107715,25
4,mSDA,e->k,electronics,kitchen,8.587715,7.628824,-0.958890,50
5,mSDA,e->k,electronics,kitchen,8.587715,10.105253,1.517538,100
6,mSDA,e->b,electronics,books,15.157879,22.560000,7.402121,25
7,mSDA,e->b,electronics,books,15.157879,26.842949,11.685070,50
8,mSDA,e->b,electronics,books,15.157879,26.368159,11.210280,100
9,mSDA,d->e,dvd,electronics,11.127778,19.871538,8.743759,25


In [15]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "inductive_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/amazon/pca/inductive_3000.csv
Resultados guardados.


# Pruebas con el dataset Twitter (2000 Dimensiones)

In [16]:
dims = 2000
dataset_name = datasets[1]

In [17]:
print tipo
print dataset_name
print dims
print data_path

pca
twitter
2000
data


In [18]:
# cargando dataset
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [19]:
parametros = {
    'n_components': [int(dims/2), int(dims/3), int(dims/4)],
}

In [20]:
porcentajes = [25, 50, 100]
modelos = {}

pairs = list(itertools.permutations(domains, 2))

for src, tgt in pairs:
    modelos[(src, tgt)] = {}
    for ptje in porcentajes:
        modelos[(src, tgt)][ptje] = [100, None]

In [21]:
df = pd.DataFrame(columns=dataframe_columns+['Porcentaje'])

i=0
j=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

porcentajes = [25, 50, 100]

for src, tgt in pairs:
    print "Tarea %d de %d" % (i+1, tareas)
    tarea = src[0]+'->'+tgt[0]
    
    #datos de entrenamiento
    X_src = labeled[src]['X_tr'][:, :dims].todense()
    y_src = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
    
    X_tgt = labeled[tgt]['X_tr'][:, :dims].todense()
    y_tgt = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
    
    X_train = np.concatenate((X_src, X_tgt))
    
    #datos de prueba
    X_tgt_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    y_tgt_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
    
    
    #baseline in-domain error
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)
    
    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tgt, y_tgt)
    b_error = 1-svc.score(X_tgt_ts, y_tgt_ts)

    k = 0    
    for n_components in parametros['n_components']:
        print "\tEntrenando modelo de adaptacion %d" % k
        # entrenar un modelo de adaptacion
        modelo_temp = PCA(n_components=n_components)
        t_adaptar = modelo_temp.fit(X_train)

        # entrenar un clasificador con los datos adaptados
        X_src_a = modelo_temp.transform(X_src)
        clf = get_best_score(X_src_a, y_src, classifier='SVC', n_jobs=4)

        # se prueba con distintos porcentajes
        j = i*3
        for porcentaje in porcentajes:
            if porcentaje == 100:
                X_test_pc, y_test_pc = X_tgt_ts, y_tgt_ts
            else:
                X_test_pc, _, y_test_pc, _ = train_test_split(X_tgt_ts, y_tgt_ts, train_size=porcentaje/float(100))
            
            X_test_pc_a = modelo_temp.transform(X_test_pc)
            t_error = 1-clf.score(X_test_pc_a, y_test_pc)
            
            if t_error <= modelos[(src, tgt)][porcentaje][0]:
                modelos[(src, tgt)][porcentaje] = [t_error, modelo_temp]
                t_loss = t_error - b_error
                df.loc[j] = ['PCA', tarea, src, tgt, b_error*100, t_error*100, t_loss*100, porcentaje]
    
            j += 1
        k += 1
    i+=1

Tarea 1 de 6
Cargando modelo existente.
tEntrenando modelo de adaptacion 0
tEntrenando modelo de adaptacion 1
tEntrenando modelo de adaptacion 2
Tarea 2 de 6
Cargando modelo existente.
tEntrenando modelo de adaptacion 0
tEntrenando modelo de adaptacion 1
tEntrenando modelo de adaptacion 2
Tarea 3 de 6
Cargando modelo existente.
tEntrenando modelo de adaptacion 0
tEntrenando modelo de adaptacion 1
tEntrenando modelo de adaptacion 2
Tarea 4 de 6
Cargando modelo existente.
tEntrenando modelo de adaptacion 0
tEntrenando modelo de adaptacion 1
tEntrenando modelo de adaptacion 2
Tarea 5 de 6
Cargando modelo existente.
tEntrenando modelo de adaptacion 0


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

tEntrenando modelo de adaptacion 1


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

tEntrenando modelo de adaptacion 2


...

/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

tEntrenando modelo de adaptacion 1


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

tEntrenando modelo de adaptacion 2


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

In [22]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss,Porcentaje
0,mSDA,r->t,rio2016,thevoice,7.362689,13.765182,6.402493,25
1,mSDA,r->t,rio2016,thevoice,7.362689,12.355212,4.992523,50
2,mSDA,r->t,rio2016,thevoice,7.362689,20.383523,13.020833,100
3,mSDA,r->g,rio2016,general,26.998127,43.194951,16.196824,25
4,mSDA,r->g,rio2016,general,26.998127,39.232334,12.234207,50
5,mSDA,r->g,rio2016,general,26.998127,42.895012,15.896885,100
6,mSDA,t->r,thevoice,rio2016,10.208711,0.699301,-9.509411,25
7,mSDA,t->r,thevoice,rio2016,10.208711,5.643739,-4.564972,50
8,mSDA,t->r,thevoice,rio2016,10.208711,13.747731,3.539020,100
9,mSDA,t->g,thevoice,general,26.998127,39.184556,12.186429,25


In [23]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "inductive_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/twitter/pca/inductive_2000.csv
Resultados guardados.
